In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
# imports
import pprint
import pickle
import glob
import random
from pathlib import Path

import torch
from tqdm import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import (
    look_at_view_transform
)
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np

from utils import utils
from utils import eval_utils
import deformation.losses as def_losses


In [17]:
evaluation_yaml_path = "data_evaluation/pix3d_chair/occnet.yaml"
#evaluation_yaml_path = "data_evaluation/shapenet_chair/occnet.yaml"
#evaluation_yaml_path = "data_evaluation/pix3d_chair/occnet_uda.yaml"

results_df = pd.read_pickle(evaluation_yaml_path.replace(".yaml","_eval_results.pkl"))
print(evaluation_yaml_path)
display(results_df)
print(results_df.mean())

data_evaluation/pix3d_chair/occnet.yaml


,2d_iou,3d_iou,chamfer_L1,gt_obj_path,input_img_path,instance,rec_obj_path
0,0.528296,0.035151,0.004484,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,2394,/home/svcl-oowl/brandon/research/sil_consisten...
1,0.421457,0.016640,0.051016,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,3729,/home/svcl-oowl/brandon/research/sil_consisten...
2,0.510172,0.406378,0.006924,/home/svcl-oowl/dataset/pix3d/model/chair/SS_1...,/home/svcl-oowl/brandon/research/sil_consisten...,1517,/home/svcl-oowl/brandon/research/sil_consisten...
3,0.487056,0.380173,0.007306,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,3018,/home/svcl-oowl/brandon/research/sil_consisten...
4,0.596261,0.146900,0.002993,/home/svcl-oowl/dataset/pix3d/model/chair/SS_0...,/home/svcl-oowl/brandon/research/sil_consisten...,0191,/home/svcl-oowl/brandon/research/sil_consisten...
...,...,...,...,...,...,...,...
3833,0.451665,0.190980,0.017660,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,3661,/home/svcl-oowl/brandon/research/sil_consisten...
3834,0.533735,0.004346,0.014498,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,1913,/home/svcl-oowl/brandon/research/sil_consisten...
3835,0.647188,0.051326,0.018265,/home/svcl-oowl/dataset/pix3d/model/chair/SS_1...,/home/svcl-oowl/brandon/research/sil_consisten...,1025,/home/svcl-oowl/brandon/research/sil_consisten...
3836,0.216825,0.006111,0.042926,/home/svcl-oowl/dataset/pix3d/model/chair/SS_1...,/home/svcl-oowl/brandon/research/sil_consisten...,1055,/home/svcl-oowl/brandon/research/sil_consisten...


2d_iou        0.500264
3d_iou        0.172550
chamfer_L1    0.019179
instance           inf
dtype: float64
